In [82]:
# from pyspark import SparkContext
# sc = SparkContext("local", "Simple App")
import random
from sklearn.model_selection import KFold
import time



In [55]:
import numpy
from scipy.stats import pearsonr


num_songs = 9724
num_users = 610


rates = [dict() for x in range(num_users)]
user_songs = [set() for x in  range(num_users)]
song_users = [set() for x in range(num_songs)]

pirs_songs = []
for x in range(num_songs):
    pirs_songs.append([-1] * num_songs)

pirs_users = []
for x in range(num_users):
    pirs_users.append([0] * num_users)

with open('new_train2.txt') as fi:
    for line in fi:
        user, song, rate = map(int, line.split()) 
        rates[user][song] = rate 
        user_songs[user].add(song) 
        song_users[song].add(user)

        


In [98]:
import math
k = 10

def gogo(train_file, test_file, num_songs, num_users):
    
    rates = [dict() for x in range(num_users)]
    user_songs = [set() for x in  range(num_users)]
    song_users = [set() for x in range(num_songs)]

    pirs_songs = []
    for x in range(num_songs):
        pirs_songs.append([-1] * num_songs)

    pirs_users = []
    for x in range(num_users):
        pirs_users.append([0] * num_users)
        
    songs_rates = []
    for x in range(num_songs):
        songs_rates.append([])

    with open(train_file) as fi:
        for line in fi:
            user, song, rate = map(int, line.split()) 
            rates[user][song] = rate 
            user_songs[user].add(song) 
            song_users[song].add(user)
            songs_rates[song].append(rate)
            
    songs_mean = [0] * num_songs
    for x in range(num_songs):
        l = len(songs_rates[x])
        if l > 0:
            songs_mean[x] = sum(songs_rates[x]) / l
        else:
            songs_mean[x] = 0
    
    
    songs_dev = [0] * num_songs
    for x in range(num_songs):
        s = 0
        for y in songs_rates[x]:
            s += (y - songs_mean[x]) ** 2
        
        if s == 0:
            break
        songs_dev[x] = (s / (len(songs_rates[x]) - 1)) ** 0.5
        
    
                    
    

    def get_mean_song(subs, i):
        a = 0
        for x in subs:
            a += rates[x][i]
        return a / len(subs)

    def get_mean_user(subs, u):
        a = 0
        for x in subs:
            a += rates[u][x]
        return a / len(subs)


    def get_pirs_song(i, j):

        subs = list(song_users[i] & song_users[j])

        if len(subs) < 2 or i == j:
            return 0

        top = 0
        down_i = 0
        down_j = 0

        i_m = get_mean_song(subs, i) 
        j_m = get_mean_song(subs, j)


        for user in subs:
            a = rates[user][i]
            b = rates[user][j]
            tmp_i = a - i_m
            tmp_j = b - j_m
            top += tmp_i * tmp_j
            down_i += tmp_i ** 2
            down_j += tmp_j ** 2

        try:
            return top / ((down_i * down_j) ** 0.5)
        except:
    #         print(i, j, [x for x in subs])
            return 0

    def get_pirs_user(u, v):

        subs = list(user_songs[u] & user_songs[v])
        if len(subs) < 2 or u == v:
            return 0

        top = 0
        down_u = 0
        down_v = 0

        u_m = get_mean_user(subs, u) 
        v_m = get_mean_user(subs, v)


        for song in subs:
            a = rates[u][song]
            b = rates[v][song]
            tmp_u = a - u_m
            tmp_v = b - v_m
            top += tmp_u * tmp_v
            down_u += tmp_u ** 2
            down_v += tmp_v ** 2

        try:
            return top / ((down_u * down_v) ** 0.5)
        except:
    #         print(u, v, [x for x in subs])
            return 0

    def res_item(u, i):
        res = songs_mean[i]
        
        tmp = [(x, get_pirs_song(x, i)) for x in user_songs[u]]
        tmp.sort(key=lambda x: x[1], reverse=True)
        tmp = tmp[:k]

        top = 0
        down = 0
        for x in tmp:
            dev_tmp = songs_dev[x[0]]
            dev_tmp = 1 if dev_tmp == 0 else dev_tmp
            top += x[1] * (rates[u][x[0]] - songs_mean[x[0]]) / dev_tmp
            down += abs(x[1])
        
        if down != 0:
            return res + songs_dev[i] * top / down
        else:
            return res
        
        
    users = set()
    songs = set()
    test_len = 0
    rmse = 0
    mae = 0
    with open(test_file) as fi:
        for x in fi:
            test_len += 1
            user, song, val = map(int, x.split())
            users.add(user)
            songs.add(song)
            q = res_item(user, song)
            
            rmse += (q - val) ** 2
            mae += abs(q - val)
            
    rmse /= test_len
    rmse = rmse ** 0.5
    
    mae /= test_len
    
    
    for user in users:
        for song in songs:
            pass
    
    return (rmse, mae)
    
    
    
data = []
with open('new_train2.txt') as fi:
    for x in fi:
        data.append(x.rstrip())


kf = KFold(5, True, 1)
for train, test in kf.split(data):
#     users = set()
#     songs = set()
    with open('train_tmp.txt', 'w') as fo:
        for x in train:
            print(data[x], file=fo)
#             user, song, _ = data[x].split()
#             users.add(user)
#             songs.add(song)
    with open('test_tmp.txt', 'w') as fo:
        for x in test:
            print(data[x], file=fo)
            
    time_s = time.time()
    print(gogo('train_tmp.txt', 'test_tmp.txt', 9724, 610), end=' ')
    print(time.time() - time_s)
    
    
    


    

(1.1922392878502568, 0.8749466777143068) 32.956669330596924
(1.1864885786528758, 0.872031631737991) 33.46180701255798
(1.1896973022744248, 0.8791783092386879) 33.367735624313354
(1.1917008405690301, 0.8740422032297741) 33.0530424118042
(1.1880419136984142, 0.87626910876604) 33.21451020240784


In [71]:
for x in range(20):
    try:
        print(x, rates[0][x], res_item(0, x))
    except:
        pass
    

#rint([x for x in user_songs[0]])

0 4 4.541457626265035
1 4 4.4071973254045655
2 4 4.183525518280833
3 5 4.591878477757143
4 5 4.582893915625734
5 3 3.2612632671909427
6 5 5.0
7 4 4.245079406741627
8 5 4.8
9 5 4.8
10 5 4.6
11 5 4.8
12 3 3.696607550482351
13 5 4.795450449395095
14 4 3.4163057716211047
15 5 3.880080673231601
16 3 4.38512482770501
17 3 4.019567625648888
18 5 4.8
19 4 4.2


In [65]:
#print([x for x in user_songs[]])
print(len(rates))

610


In [ ]:
pirs_songs = pirs_songs.map(lambda x: list(map()))

In [95]:
# s = 0
# g = 0
# f = 0

    
for x in range(num_users):
    pirs_users[x][x] = -2
    for y in range(x + 1, num_users):
        f += 1
        pirs = get_pirs_user(x, y)
        pirs_users[x][y] = pirs
        pirs_users[y][x] = pirs
            
#         if -1 <= pirs <= 1:
#             g += 1
#         else:
#             s += 1
        
        
            

        


In [94]:
# print(rates[946][18], rates[946][237])
# print(rates[932][18], rates[932][237])
s
print(610 * 610 // 2, s, g, f, s + g)

186050 57787 127958 371490 185745


In [36]:
from scipy.stats import pearsonr
import numpy

i, j = 1, 11
subs = song_users[i] & song_users[j]
ar1 = [rates[x][i] for x in subs]
ar2 = [rates[x][j] for x in subs]
print(pearsonr(ar1, ar2))
print(numpy.corrcoef(ar1, ar2)[0, 1])

(nan, nan)
nan


In [97]:
from pyspark import SparkContext
sc = SparkContext("local", "Simple App")
data = sc.textFile('new_train2.txt')

In [106]:
data = data.map(lambda x: list(map(int, x.split())))

In [108]:
for x in data.top(10):
    print(x)

[609, 9723, 3]
[609, 9722, 3]
[609, 9721, 3]
[609, 9720, 4]
[609, 9719, 2]
[609, 9718, 3]
[609, 9717, 3]
[609, 9716, 3]
[609, 9715, 3]
[609, 9714, 3]
